In [19]:
import pyodbc
import pandas as pd
import numpy as np

#For python3 MSSQL

cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"   #For Connection
                      "Server=server_name;"
                      "Database=database_name;"
                      "UID=login;"
                      "PWD=password;")
df_raw = pd.read_sql_query("SELECT * \
                        FROM [dbo].[Table_Name]", cnxn)

In [20]:
df_raw.drop(['Organic_Frequency',
       'Paid_Frequency', 'CPM',
       'Total_Impressions','Total_Frequency',
        'CTR', 'CPC', 
       'Number_of_engagements',
       'Engagement_rate', 'CPE','CPView_3_sec',
       'CPView_10_sec', 'CPView_video_completions'], axis = 1, inplace = True)

In [21]:
def cleaning_function(dataframe):
    
    # Checking for None, '0' and '#DIV/0' values
    if dataframe[dataframe == '#DIV/0!'].count().sum() > 0:
        dataframe.replace('#DIV/0!', 0, inplace = True)
        
    elif dataframe[dataframe == '0'].count().sum() > 0:
         dataframe.replace('0!', 0, inplace = True)
            
    elif dataframe.isnull().sum().sum() > 0:
        dataframe.fillna(value=np.nan, inplace=True)
    
    # Cleaning 'Cost' column by deleting Kc and ',' to convert it into float later
    dataframe['Cost'] =  dataframe['Cost'].str.replace(' Kč', '')
    dataframe['Cost'] =  dataframe['Cost'].str.replace(',', '')

    #dataframe.dropna(axis=1, how='all', inplace = True)
    
    # Creating lists of future float and integer columns
    list_of_float_columns_before_calculations = ['Cost']  #delete 'Cost' after automation     
    list_of_integer_columns_before_calculations = ['Měsíc', 'Organic_Impressions', 'Organic_Reach', 
                               'Paid_Impressions', 'Paid_Reach', 'Total_Reach', 
                               'Number_of_link_clicks', 'Likes_reactions', 'Shares',
                               'Comments', 'Saves', 'Others_Event_reponses_Offer_claims',
                               '_3_sec_Video_views', '_10_sec_Video_views','Video_completions']

    # Filling out empty cells by 0 in future float and integer columns
    for col in dataframe[list_of_integer_columns_before_calculations]:
            dataframe[col].replace(np.nan, 0, inplace = True)
            
    for col in dataframe[list_of_float_columns_before_calculations]:
            dataframe[col].replace(np.nan, 0, inplace = True)
    
    # Changing the types of columns to actual float and integer
    dataframe[list_of_float_columns_before_calculations] = dataframe[list_of_float_columns_before_calculations].astype(float)
    dataframe[list_of_integer_columns_before_calculations] = dataframe[list_of_integer_columns_before_calculations].astype(int)
    
    # Creating new columns by calculations
    dataframe['Organic_Frequency'] = [org_impr /  org_reach if org_reach != 0 else 0 for org_impr, org_reach in zip(dataframe['Organic_Impressions'], dataframe['Organic_Reach'])]
    
    dataframe['Paid_Frequency'] = [paid_impr /  paid_reach if paid_reach != 0 else 0 for paid_impr, paid_reach in zip(dataframe['Paid_Impressions'], dataframe['Paid_Reach'])]
    
    #dataframe['Cost'] = dataframe['Cost'] * 28 - will be used, when we will have automated push to warehouse, so the metric will be in EUR in the beginning
    
    dataframe['CPM'] = [cost / (paid_impr / 1000) if paid_impr != 0 else 0 for cost, paid_impr in zip(dataframe['Cost'], dataframe['Paid_Impressions'])]
    
    dataframe['CPU'] = [cost / paid_reach if paid_reach != 0 else 0 for cost, paid_reach in zip(dataframe['Cost'], dataframe['Paid_Reach'])]
    
    dataframe['Total_Impressions'] = dataframe['Organic_Impressions'] + dataframe['Paid_Impressions']
                        
    dataframe['Total_Frequency'] = [tot_impr / tot_reach if tot_reach != 0 else 0 for tot_impr, tot_reach in zip(dataframe['Total_Impressions'], dataframe['Total_Reach'])]
                        
    dataframe['CTR'] = [n_clicks / paid_impr if paid_impr != 0 else 0 for n_clicks, paid_impr in zip(dataframe['Number_of_link_clicks'], dataframe['Paid_Impressions'])]
                                    
    dataframe['CPC'] = [cost / n_clicks if n_clicks != 0 else 0 for cost, n_clicks in zip(dataframe['Cost'], dataframe['Number_of_link_clicks'])]
                        
    #dataframe['Others_Event_reponses_Offer_claims'] = dataframe['Others_Event_reponses'] + dataframe['Offer_claims'] -  will be used, when we will have automated push to warehouse, so the metric will be in EUR in the beginning
                        
    dataframe['Number_of_engagements'] = dataframe['Likes_reactions'] + dataframe['Shares'] + dataframe['Comments'] + dataframe['Saves'] + dataframe['Others_Event_reponses_Offer_claims']
                        
    dataframe['Engagement_rate'] = [n_eng / paid_impr if paid_impr != 0 else 0 for n_eng, paid_impr in zip(dataframe['Number_of_engagements'], dataframe['Paid_Impressions'])]
                        
    dataframe['CPE'] = [cost / n_eng if n_eng != 0 else 0 for cost, n_eng in zip(dataframe['Cost'], dataframe['Number_of_engagements'])]
                        
    dataframe['CPView_3_sec'] =  [cost / three_sec if three_sec != 0 else 0 for cost, three_sec in zip(dataframe['Cost'], dataframe['_3_sec_Video_views'])]
    
    dataframe['CPView_10_sec'] =  [cost / ten_sec if ten_sec != 0 else 0 for cost, ten_sec in zip(dataframe['Cost'], dataframe['_10_sec_Video_views'])]
                        
    dataframe['CPView_video_completions'] = [cost /  completions if completions != 0 else 0 for cost, completions in zip(dataframe['Cost'], dataframe['Video_completions'])]


                                                                          
    # Creatin a list of float columns to round them 
    list_of_new_float_columns = ['Organic_Frequency', 'Paid_Frequency', 'CPM', 'CPU', 'Total_Frequency', 'CTR', 
                                  'CPC', 'Engagement_rate', 'CPE', 'CPView_3_sec', 'CPView_10_sec', 'CPView_video_completions']      
    
    # Rounding values in newly created columns
    dataframe[list_of_new_float_columns] = dataframe[list_of_new_float_columns].round(decimals=2)
    
    # Replacing columns, so they have original order
    original_order = ['Měsíc', 'Datum_zveřejnění_postu', 'Post_ID', 'Preview',
       'Text_Postu', 'Format', 'Typ_postu', 'Activity_Type', 'Objective',
       'Organic_Impressions', 'Organic_Reach', 'Organic_Frequency',
       'Paid_Impressions', 'Paid_Reach', 'Paid_Frequency', 'Cost', 'CPM',
       'CPUU', 'Total_Impressions', 'Total_Reach', 'Total_Frequency',
       'Column_22', 'Number_of_link_clicks', 'CTR', 'CPC', 'Column_26',
       'Likes_reactions', 'Shares', 'Comments', 'Saves',
       'Others_Event_reponses_Offer_claims', 'Number_of_engagements',
       'Engagement_rate', 'CPE', 'Column_35', '_3_sec_Video_views',
       '_10_sec_Video_views', 'Video_completions', 'CPView_3_sec',
       'CPView_10_sec', 'CPView_video_completions']
    dataframe = dataframe[original_order]
    
    dataframe.to_excel('Output.xlsx', index = False)